In [22]:
import os
import sys
import scanpy as sc
pwd = '/home/lijiahao/workbench/SAVE'
os.chdir(pwd)
sys.path.append(pwd)

In [23]:
from model.utils.process_h5ad import preprocessing_rna, batch_scale

In [ ]:
# data can be download at: https://datasets.cellxgene.cziscience.com/99040b08-7e1a-4d81-911c-4d2fd2335757.h5ad
adata = sc.read_h5ad('data/cancer_lung/cancer_lung.h5ad')

In [ ]:
cond_cols = ['assay', 'cell_type', 'development_stage', 'disease', 'uicc_stage']
adata.obs['batch'] = adata.obs['assay']

In [ ]:
adata.obs['development_stage']

In [ ]:
test_cond1 = {'development_stage': '53-year-old human stage', 'uicc_stage': 'IV'}

test_cond2 = {'development_stage': '78-year-old human stage', 'uicc_stage': 'III'}

In [ ]:
adata = preprocessing_rna(adata, is_batch_scale=False)
adata.write_h5ad('data/cancer_lung/cancer_lung_hvg.h5ad')

In [ ]:
from functools import reduce
import operator

def get_cond_data(adata, input_cond):
    msks = [adata.obs[x[0]] == x[1] for x in input_cond.items()]
    total_msk = reduce(operator.and_, msks)
    return adata[total_msk].copy()

In [ ]:
msks_1 = [adata.obs[x[0]] == x[1] for x in test_cond1.items()]
total_msk1 = reduce(operator.and_, msks_1)
msks_2 = [adata.obs[x[0]] == x[1] for x in test_cond2.items()]
total_msk2 = reduce(operator.and_, msks_2)

In [ ]:
total_test_msk = total_msk1 | total_msk2

In [ ]:
test_data = adata[total_test_msk].copy()
train_data = adata[~total_test_msk].copy()

In [ ]:
test_data.write_h5ad('data/cancer_lung/cancer_lung_hvg_test.h5ad')
train_data.write_h5ad('data/cancer_lung/cancer_lung_hvg_train.h5ad')